### Install / Import 모듈 

|   
|:--| 
|<img src ='https://bit.ly/39XPeCc' width=200 align='left'>
|<h1>    starcraft_XGBM_0415_startover_r01.ipynb
</h1>    

### **데이터 분리** (parametric study)
> - 80% 20%  Train/Test 자체 Test Data set 으로 모델을 검증 한다.
> - Sklearn.train_data_split() 을 이용해서 분리한다.
> - 파라메터를 바꿔 가면서 모델의 예측율을 사전 검증

In [0]:
from google.colab import drive 
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
# 코랩에 없는 라이브러리 설치해주기
!pip install bayesian-optimization

In [0]:
"""
# WAY-01 = LGBM, WAY-02 = XGBM
"""

init_points, n_iter = (50, 20)                      
filename_submit = f"submission_0415_XGBM_{init_points}_{n_iter}_startover_r01.csv"
file_save = False

## 사용함수의 정의

In [0]:
""" 
# 기본적인 module import 위치로 작업폴더 변경 getcwd() --> chidr()
# HOME 을 지정하고 작업폴더를 HOME 으로 변경.

# # for PC
# import os
# HOME= 'dacon_competition_2020'
# dir_base = "".join(os.getcwd().partition(HOME)[:2]) + "\\"
# os.chdir(dir_base)  
"""   

# for gDrive
import os
HOME = '/content/drive/My Drive/Colab Notebooks/'
os.chdir(HOME)

In [0]:
from _assets.config_dirs_gDrive import *
from _assets.modules import *
from _assets.module_xgbm_model import *


# FOR gDrive Colab - 화일분석에 필요한 공동폴더를 등록합니다.
# - echo = True : SYS.PATH INSERT 상황 보여줌


# OS 화일 및 DF 정보조회를 위한 탐색 모듈


# XGBM_CV 모델 - by SongDo_StudyGroup Code 참조



Using TensorFlow backend.


In [0]:
import os
import pandas as pd                         # 데이터 분석 라이브러리
import numpy as np                          # 계산 라이브러리
import matplotlib.pyplot as plt             # * 그래프 이미지

from tqdm import tqdm                       # 진행바

from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings           
import random                       
warnings.filterwarnings("ignore")           # 경고 문구 미표시

## 데이터를 불러온다

In [0]:
%%time
"""
# show_infoDF_from(x_train) # *** DATA SHAPE = [ 38,872 x 1,579 ] 
# to <class 'pandas.core.frame.DataFrame'>
# show_infoDF_from(y_train) # *** DATA SHAPE = [ 38,872 x 3 ]
# to <class 'numpy.ndarray'> ...  array([1, 1, 0, ..., 0, 1, 0])
"""
# x_train = pd.read_csv(dir_base + raw + 'xtra_remake_xtrain_final.csv')  
# ytrain = pd.read_csv(dir_base + remake + 'df_ytrain_remake.csv')  
# y_train = ytrain['winner'].values    


_df1 = pd.read_csv(dir_base + raw + 'X1_train.csv')

x_train = _df1.iloc[:,:-1]              # pd.DataFrame
y_train = _df1.loc[:, 'Y'].to_numpy()   # np.array  

CPU times: user 73 ms, sys: 31.2 ms, total: 104 ms
Wall time: 2.67 s


In [0]:
# x_train.shape, y_train.shape  # ((31097, 27), (31097,))
# type(y_train)                 # numpy.ndarray
# y_train                       # array([1, 1, 0, ..., 1, 1, 0])

### Data 전처리 및 저장하기
- 전처리 된 x_train, y_train 내용을 확인하고, 저장한다.

In [0]:
# 화일위치 / list 확인하기 ... 사용함수
# show_ls('/content')          # drive 가 붙었는지 확인!
show_ls(dir_base + raw)      # 불러올 Data 화일이름 확인!

False
DIR_TARGET=/content/drive/My Drive/Colab Notebooks
/content/drive/My Drive/Colab Notebooks/competition/c03_starcraft_prediction/data_raw/
----------------------------------------
  01. README_data_raw.md
  02. X1_test.csv
  03. X1_train.csv
  04. X2_test.csv
  05. X2_train.csv
  06. sample_submission.csv
  07. test.csv
  08. train.csv


## XGBM Classifier 모델 적용
 - 송도 머신러닝 Sturdy 유창준님 파라메터 모델 공유 받음!
 -  Extreme Gradient Boost Method 적용 - [출처] :  YW & YY's Python, Machine & Deep Learning

In [0]:
%%time

var_fixed = partial(                       # 모델 그 외 변수는 고정
        XGB_cv, 
        x_data=x_train, 
        y_data=y_train, 
        n_splits=5, 
        output='score'
    )

XGBo = BayesianOptimization(                # 베이지안 최적화 범위 설정
        var_fixed, 
        {
            # 'colsample_bynode':(0, 1),    # unexpected argument in XGBM
            'colsample_bytree': (0.5, 1),   # 0.5 ~ 0.8
            'learning_rate': (0.1, 0.6),    # 0.6xxx
            # 'n_estimators' : (16, 62),    # unexpected argument in XGBM
            # 'num_leaves' : (16, 62),      # unexpected argument in XGBM
            'reg_alpha' : (7, 20),          # L1 - 7 ~ 20
            'reg_lambda' : (8, 50),         # L2 - 8 ~ 45
            'subsample': (0.5, 0.9),        # 0.5 ~ 0.7
         

            'max_depth': (8, 512),          # ~ 500
            'max_delta_step' : (1, 25),     # 8.x
            'gamma' : (5, 20),              # 1 ~ 15
        },
        # random_state = random.randrange(50000) # 유동 시드 1~50000, 초기값
        random_state = random.randrange(50000) # 시드 고정 = 4321
    )

# XGBo.maximize(init_points=15, n_iter=60)  # 처음 15회 랜덤값으로 score 계산 후 60회 최적화 = 4/04(토)
XGBo.maximize(init_points, n_iter)

|   iter    |  target   | colsam... |   gamma   | learni... | max_de... | max_depth | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.637    |  0.7742   |  14.71    |  0.2321   |  17.25    |  363.9    |  19.62    |  12.76    |  0.8955   |
|  2        |  0.6306   |  0.6162   |  19.42    |  0.3904   |  7.261    |  256.4    |  10.37    |  42.0     |  0.5312   |
|  3        |  0.6417   |  0.9804   |  6.588    |  0.1989   |  23.69    |  251.4    |  8.492    |  40.21    |  0.5392   |
|  4        |  0.6408   |  0.9462   |  9.748    |  0.1332   |  1.833    |  308.2    |  10.7     |  42.71    |  0.5824   |
|  5        |  0.6359   |  0.9861   |  16.6     |  0.4328   |  1.746    |  344.5    |  12.81    |  24.37    |  0.678    |
|  6        |  0.6385   |  0.6523   |  11.56    |  0.3019   |  13.17    |  272.7    |  12.36    |  27.17    |  0.6759   |
|  7        |  0.6333   

### 3.0 모델 학습 및 검증
- Model Tuning & Evaluation

> 1. AUC가 가장 높은 하이퍼 파라미터를 사용해 최종 모델을 얻습니다.
> 1. 훈련 세트와 같은 방법으로 테스트 세트에서 Feature를 추출합니다.
> 1. 최종 모델을 사용해 예측을 수행합니다.
> 1. 예측 결과를 submission.csv로 저장합니다.

| ITER | TARGET | COLSAMPLE | GAMMA | LEARN_RATE | MAX_DELTA | MAX_DEPTH | REG_ALPHA | REG_LAMBDA | SUBSAMPLE |
|--|--|--|--|--|--|--|--|--|--|
|  1        |  0.637    |  0.7742   |  14.71    |  0.2321   |  17.25    |  363.9    |  19.62    |  12.76    |  0.8955   |
|  2        |  0.6306   |  0.6162   |  19.42    |  0.3904   |  7.261    |  256.4    |  10.37    |  42.0     |  0.5312   |
|  3        |  0.6417   |  0.9804   |  6.588    |  0.1989   |  23.69    |  251.4    |  8.492    |  40.21    |  0.5392   |
|  4        |  0.6408   |  0.9462   |  9.748    |  0.1332   |  1.833    |  308.2    |  10.7     |  42.71    |  0.5824   |
|  5        |  0.6359   |  0.9861   |  16.6     |  0.4328   |  1.746    |  344.5    |  12.81    |  24.37    |  0.678    |
|  6        |  0.6385   |  0.6523   |  11.56    |  0.3019   |  13.17    |  272.7    |  12.36    |  27.17    |  0.6759   |
|  7        |  0.6333   |  0.5475   |  14.89    |  0.4889   |  8.224    |  453.4    |  19.26    |  10.5     |  0.7891   |
|  8        |  0.6421   |  0.9586   |  8.6      |  0.215    |  16.62    |  16.49    |  8.444    |  35.57    |  0.5633   |
|  9        |  0.6409   |  0.6151   |  9.27     |  0.1605   |  3.964    |  224.3    |  14.69    |  10.75    |  0.6354   |
|  10       |  0.6422   |  0.6098   |  7.758    |  0.425    |  21.5     |  193.8    |  10.29    |  42.67    |  0.8968   |
|  11       |  0.6353   |  0.9438   |  18.59    |  0.2033   |  13.22    |  10.8     |  19.54    |  15.68    |  0.6975   |
|  12       |  0.6431   |  0.9847   |  6.438    |  0.1648   |  9.992    |  171.0    |  8.58     |  46.7     |  0.5244   |
|  13       |  0.6344   |  0.682    |  18.43    |  0.2585   |  10.11    |  415.0    |  15.61    |  47.38    |  0.6662   |
|  14       |  0.6406   |  0.7198   |  8.694    |  0.3409   |  14.11    |  352.0    |  11.72    |  49.44    |  0.7835   |
|  15       |  0.6381   |  0.9125   |  10.76    |  0.3368   |  19.18    |  191.0    |  19.37    |  27.02    |  0.5894   |
|  16       |  0.6363   |  0.8853   |  17.32    |  0.3217   |  12.2     |  177.1    |  16.59    |  39.75    |  0.7485   |
|  17       |  0.6369   |  0.9016   |  14.5     |  0.171    |  4.717    |  360.9    |  14.99    |  9.614    |  0.5328   |
|  18       |  0.6376   |  0.8326   |  16.37    |  0.406    |  16.16    |  190.9    |  8.965    |  9.143    |  0.8536   |
|  19       |  0.6411   |  0.7981   |  9.339    |  0.2919   |  24.05    |  261.0    |  11.92    |  49.65    |  0.6929   |
|  20       |  0.6318   |  0.6301   |  15.85    |  0.4815   |  5.284    |  356.4    |  16.48    |  25.2     |  0.5646   |

# XGBM Parametric Study

| NUM | SEED | RANDOM | ITERATION | n_ESTIMATION | num_LEAF | TIME_SPEND| PREDICTION |
|:---:|-----:|---:|---:|-----:|-----:|----------:|:--------:|
|  1  | 50000|    |    |      |      | 1.45.21 s | 60.875 % |
|     | 50000|    |    |      |      | -.--.-- s | --.000 % |
|     | 50000|    |    |      |      | -.--.-- s | --.000 % |
|     | 50000|    |    |      |      | -.--.-- s | --.000 % |

In [0]:
"""
# xgb.cv: Cross Validation documentation
# https://rdrr.io/cran/xgboost/man/xgb.cv.html
"""

params = XGBo.max['params']

models = XGB_cv(                    # cv = cross validation
        max_depth =        params['max_depth'], 
        learning_rate =    params['learning_rate'],
        subsample =        params['subsample'], 
        colsample_bytree = params['colsample_bytree'], 
        reg_alpha =        params['reg_alpha'], 
        reg_lambda =       params['reg_lambda'],
        gamma =            params['gamma'],
        max_delta_step =   params['max_delta_step'],
        x_data=x_train, 
        y_data=y_train, 
        n_splits=5, 
        output='model'
    )

# TEST를 풀기위해, 학습데이터로 변경 (data_preps)
 1. Train Data set = [ 67,091,776 x 7 ] ... 70%
  - game_id 별로 정리하면 [38,872 x 27]로 압축된다. 
  - 경기당, 평균 1,726개의 이벤트가 존재한다.(압축률 0.057939 %)
 > - x_train set = *** DATA SHAPE = [ 38,872 x 27 ] pandas.DataFrame
 > - y_train set = *** DATA SHAPE = [ 38,872 x 1 ]  np.array
 
 1. Test Data set  = [ 28,714,849 x 6 ] ... 30%
  - game_id 별로 정리하면 [16,787 x 27] 로 압축된다. 
  - 경기당 평균 1711개의 이벤트가 존재한다.(압축률 0.058461 %)
 > - x_test set = *** DATA SHAPE = [16,787 x 27] pandas.DataFrame 
 > - y_test set = *** DATA SHAPE = [16,787 x 1] np.array =  대회주관자가 가지고 있음
 
 1. Submission = y_test set 
  - submission = [16,787 x 1] np.array to [16,787 x 2 ] pandas.DataFrame --> df.csv

In [0]:
"""
# 테스트용 데이터를 전처리 해서 분석 준비 ... (시간소요) 미리 저장한 화일을 불러온다
# 저장했으면 비활성 처리
# x_test, _ = data_preparation(test, answer=False)                 # [ 28,714,849 x 6 ] ... 30%
# x_test.to_csv(dir_base + remake + 'df_xtest_remake.csv')
# x_test = pd.read_csv(dir_base + remake + 'df_xtest_remake.csv')  # [  16,787 x 27 ]
"""
# x_test = pd.read_csv(dir_base + remake + 'xtra_remake_xtest_final.csv')    # *** DATA SHAPE = [ 16,787 x 27 ]

_df2 = pd.read_csv(dir_base + raw + 'X1_test.csv')
x_test = _df2.iloc[:,:-1]
y_test = _df2.loc[:, 'Y']

In [0]:
x_test.shape, y_test.shape  # ((31097, 27), (31097,))
type(y_test)                 # numpy.ndarray
y_test                       # array([1, 1, 0, ..., 1, 1, 0])

0       0
1       1
2       0
3       0
4       0
       ..
7770    1
7771    0
7772    0
7773    1
7774    0
Name: Y, Length: 7775, dtype: int64

# 예측결과 Submission 화일 만들기

In [0]:
"""
# pd.read_csv('data/sample_submission.csv', index_col=0)
# sample_submission = pd.read_csv(dir_base + sub_base + "sample_submission.csv")
"""

preds = []

for model in models:
    pred = model.predict_proba(x_test)[:, 1]
    preds.append(pred)
    
pred = np.mean(preds, axis=0)

In [0]:
"""
# submission = pd.read_csv(dir_base + raw + "sample_submission.csv", index_col='game_id') 
# submission = pd.DataFrame(pred, columns=['winner'])

# submission['winner'] = pred
"""


submission = pd.DataFrame(pred, columns=['winner'])
# submission.to_csv(dir_base + submit + filename_submit, index=False)

In [0]:
"""
# submission = pd.read_csv(dir_base + raw + "sample_submission.csv") 
# submission['winner'] = pred
"""
if file_save:
    submission.to_csv(dir_base + submit + filename_submit, index=False)
else:
    print('*** SAVING FILE SKIPPED!! ***')    

*** SAVING FILE SKIPPED!! ***


In [0]:
"""
#  자체 스코어를 파악해 본다 = 정답률 60.37 % 
# - True     4694
# - False    3081
"""

df_score =  submission.copy()             # deep copy
df_score['Y-pred'] = df_score.winner.apply(lambda x: 1 if x > 0.5 else 0)
df_score['Y'] = y_test
df_score['check-sum'] = (df_score['Y-pred'] == df_score['Y'])

_ = df_score['check-sum'].value_counts().to_list()
score = _[0]/(_[0]+_[1])
print(f"*** SCORE = {score * 100:0.7} %")

*** SCORE = 60.8746 %


# 결과 확인하기
- 3월 데이콘 대회 홈페이지 = https://bit.ly/39bqWVg
- 결과 제출하기 = https://dacon.io/competitions/official/235583/mysubmission/

In [0]:
submission

,winner
0,0.424345
1,0.639418
2,0.340709
3,0.364041
4,0.765779
...,...
7770,0.399698
7771,0.361303
7772,0.587917
7773,0.732650


In [0]:
df_score.head(20)

,winner,Y-pred,Y,check-sum
0,0.424345,0,0,True
1,0.639418,1,1,True
2,0.340709,0,0,True
3,0.364041,0,0,True
4,0.765779,1,0,False
5,0.549900,1,0,False
6,0.406670,0,0,True
7,0.530096,1,0,False
8,0.756660,1,1,True
9,0.551271,1,0,False
